Clase 7
=====

Objetivos:

* Uso avanzado de cobrapy
* Análisis de soluciones alternativas de FBA


Solución tarea optativa, cortesía de Maximiliano Mellado
------------

Simula y los flujos metabólicos de *E. coli* cuando:
    
1. El flujo de consumo de glucosa ("EX_glc__D_e") es igual a -2 [mmol/gDW/h]
2. El limite inferior para el flujo de consumo de oxigeno ("EX_o2_e") es igual a -10 [mmol/gDW/h]

En cada caso ilustra la distribución de flujos usando ```escher```

In [1]:
import cobra
import cobra.test
model = cobra.test.create_test_model("ecoli")
model.reactions.get_by_id("EX_glc__D_e").bounds=(-2,-2)
solution=model.optimize()
model.summary()

import escher
from escher import Builder
# VISUALIZATION OF RESULTS
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)
builder.reaction_data = solution.fluxes
builder

Builder(reaction_data={'DM_4crsol_c': 4.148296729398152e-05, 'DM_5drib_c': 4.185501184370257e-05, 'DM_aacald_c…

In [2]:
import cobra
import cobra.test
model = cobra.test.create_test_model("ecoli")
model.reactions.get_by_id("EX_glc__D_e").bounds=(-10,-10)
model.reactions.get_by_id("EX_o2_e").bounds=(-10,-10)
solution=model.optimize()
model.summary()

import escher
from escher import Builder
# VISUALIZATION OF RESULTS
builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core',
)
builder.reaction_data = solution.fluxes
builder

Builder(reaction_data={'DM_4crsol_c': 0.00015852414858272547, 'DM_5drib_c': 0.00015994588982562452, 'DM_aacald…

Uso avanzado de cobrapy
==========

Construcción de una red metabólica
----------

Ahora construiremos nuestra propia red metaolica usando como ejemplo la red metanogénica que vimos en la clase pasada:

<img src="https://raw.githubusercontent.com/modcommet/Clases/master/metanogenesis.png" width="400">


In [3]:
from cobra import Model, Reaction, Metabolite
cobra_model = Model('primordialMetabolism')

# Ractions
E1 = Reaction('E1')
E1.name = 'Exchange reaction of CO2'
E1.subsystem = 'Entrance'
E1.lower_bound = 0.  
E1.upper_bound = 10.  


E2 = Reaction('E2')
E2.name = 'Exchange reaction of H2'
E2.subsystem = 'Entrance'
E2.lower_bound = 0.  
E2.upper_bound = 1000.  


R1 = Reaction('R1')
R1.name = 'First reaction'
R1.subsystem = 'Internal reaction'
R1.lower_bound = 0.  
R1.upper_bound = 1000. 
R2 = Reaction('R2')
R2.name = 'Second reaction'
R2.subsystem = 'Internal reaction'
R2.lower_bound = 0.  
R2.upper_bound = 1000.  

R3 = Reaction('R3')
R3.name = 'Third reaction'
R3.subsystem = 'Internal reaction'
R3.lower_bound = 0.  
R3.upper_bound = 1000. 

E3 = Reaction('E3')
E3.name = 'Exchange reaction of CH4'
E3.subsystem = 'Exit'
E3.lower_bound = 0.  
E3.upper_bound = 1000.  

E4 = Reaction('E4')
E4.name = 'Exchange reaction of F420'
E4.subsystem = 'Exit'
E4.lower_bound = 0. 
E4.upper_bound = 1000. 

E5 = Reaction('E5')
E5.name = 'Exchange reaction of H20'
E5.subsystem = 'Exit'
E5.lower_bound = 0.  
E5.upper_bound = 1000. 


# Metabolites
CO2 = Metabolite('CO2', formula='CO2',name='carbon dioxide',compartment='c')
H2 = Metabolite('H2', formula='H2',name='H2',compartment='c')
F420 = Metabolite('F420', formula='F420',name='F420',compartment='c')
I1 = Metabolite('I1', formula='I1',name='I1',compartment='c')
H2O = Metabolite('H2O', formula='H2O',name='H2O',compartment='c')
CH4 = Metabolite('CH4', formula='CH4',name='CH4',compartment='c')


# Agrega el resto de los metabolitos


E1.add_metabolites({CO2: 1.0})
E2.add_metabolites({H2: 1.0})
E3.add_metabolites({CH4: -1.0})
E4.add_metabolites({F420: -2})
E5.add_metabolites({H2O: -1})
R1.add_metabolites({CO2:-1,H2: -1.0,F420:-1,I1:1})
R2.add_metabolites({H2:-1,F420:1,I1:-1,CH4:1})
R3.add_metabolites({F420:1,I1:1,CH4:-1,H2O:1})



# ya sabes que hacer 
cobra_model.add_reaction(E1)
cobra_model.add_reaction(E2)
cobra_model.add_reaction(R1)
cobra_model.add_reaction(R2)
cobra_model.add_reaction(R3)
cobra_model.add_reaction(E3)
cobra_model.add_reaction(E4)
cobra_model.add_reaction(E5)


# lo mismo aquí

cobra_model.objective = 'E3'

In [4]:
# Iterate through the the objects in the model
print("Reactions")
print("---------")
for x in cobra_model.reactions:
    print("%s : %s" % (x.id, x.reaction))
print("Metabolites")
print("-----------")
for x in cobra_model.metabolites:
    print('%s : %s' % (x.id, x.formula))

Reactions
---------
E1 :  --> CO2
E2 :  --> H2
R1 : CO2 + F420 + H2 --> I1
R2 : H2 + I1 --> CH4 + F420
R3 : CH4 --> F420 + H2O + I1
E3 : CH4 --> 
E4 : 2 F420 --> 
E5 : H2O --> 
Metabolites
-----------
CO2 : CO2
H2 : H2
F420 : F420
I1 : I1
CH4 : CH4
H2O : H2O


In [5]:
solution = cobra_model.optimize()
print("Valor función objetivo: ", solution.objective_value )
for i in cobra_model.reactions:
    print( i.id, i.x )

Valor función objetivo:  10.0
E1 10.0
E2 20.0
R1 10.0
R2 10.0
R3 0.0
E3 10.0
E4 0.0
E5 0.0


Comparemos el resultado con numpy, cortesía de Franz Winklmeier y Maximiliano Mellado
-------------------------------

In [6]:
import numpy as np
from scipy.optimize import linprog
from numpy.linalg import solve

# E1,E2,E3,E4,E5,R1,R2,R3

A = np.array([
[1,0,0,0,0,-1,0,0],  # CO2
[0,1,0,0,0,-1,-1,0], # H2
[0,0,0,0,0,1,-1,1],  # I2
[0,0,0,-2,0,-1,1,1], # F420
[0,0,-1,0,0,0,1,-1], # CH4
[0,0,0,0,-1,0,0,1]]) # H2O       

b = np.array([0,0,0,0,0,0])
c = np.array([0,0,1,0,0,0,0,0])

LB=[0,0,0,0,0,0,0,0]       # [0,0,0,0,0,0]
UB=[10,1000,1000,1000,1000,1000,1000,1000]     # ya sabes la idea

res = linprog(-c, A_eq=A, b_eq=b, bounds=list(zip(LB,UB)))
print('Optimal value:', round(-res.fun) )
for i,reactionName in enumerate(["E1","E2","E3","E4","E5","R1","R2","R3"]):
    print(reactionName, round(res.x[i]))


Optimal value: 10.0
E1 10.0
E2 528.0
E3 10.0
E4 508.0
E5 508.0
R1 10.0
R2 518.0
R3 508.0


Ejercicio
---------

¿Cómo se puede explicar que numpy y cobrapy entregan diferentes soluciónes?

Análisis de soluciones alternativas de FBA
======

Flux variability análisis
--------

FBA no siempre da una solución única porque multiples configuraciones de flujos pueden alcanzar el mismo valor de la función objetivo. FVA (flux variability análisis) encuentra el rango de cada flujo metabólico en el valor óptimo.

In [7]:
from cobra.flux_analysis import flux_variability_analysis
flux_variability_analysis(cobra_model, cobra_model.reactions[:8])

,minimum,maximum
E1,10.0,10.0
E2,20.0,1000.0
R1,10.0,10.0
R2,10.0,990.0
R3,0.0,980.0
E3,10.0,10.0
E4,0.0,980.0
E5,0.0,980.0


Flux sampling
-------

Otra alternativa para explorar el espacio de soluciones alternativas de FBA es tomar muestrasa aleatorias. Esto se llama Flux sampling.

In [8]:
from cobra.test import create_test_model
from cobra.sampling import sample
s = sample(cobra_model, 100) # tomaremos 100 muestras aleatorias
s.head(6)

,E1,E2,R1,R2,R3,E3,E4,E5
0,5.578858,703.469075,5.578858,697.890217,692.311360,5.578858,692.311360,692.311360
1,3.154628,617.856314,3.154628,614.701686,611.547059,3.154628,611.547059,611.547059
2,1.309241,58.808974,1.309241,57.499733,56.190492,1.309241,56.190492,56.190492
3,7.577828,813.808531,7.577828,806.230703,798.652875,7.577828,798.652875,798.652875
4,7.539349,370.522077,7.539349,362.982728,355.443378,7.539349,355.443378,355.443378
5,6.888347,91.041822,6.888347,84.153475,77.265128,6.888347,77.265128,77.265128


In [9]:
#Si quiero ver solo los valores de la reacción E1
s["E1"]

0     5.578858
1     3.154628
2     1.309241
3     7.577828
4     7.539349
        ...   
95    3.574605
96    4.835266
97    4.213900
98    3.012304
99    5.611660
Name: E1, Length: 100, dtype: float64

In [10]:
#Si quiero ver el valor promedio de la reacción E1 a lo largo de las 100 muestras
s["E1"].mean()

5.214572156821046

Ejercicios
==========

1. Computa FBA de la red de *Escherichia coli*
2. Realiza FVA para la solución anterior.
3. Computa 100 muestras aleatorias del espacio de soluciones alternativas de 1.